In [1]:
from dataclasses import dataclass

from agent_prompts import PROFILE_CREATION_PROMPT
from models import UserProfile
from pydantic_ai import Agent
from rich.prompt import Prompt

from knd.ai import run_until_completion


In [2]:
MODEL = "google-gla:gemini-1.5-flash"


@dataclass
class FinDeps:
    user_profile: UserProfile | None = None


user_profile_creator = Agent(
    MODEL,
    result_type=UserProfile | str,  # type:ignore
    deps_type=FinDeps,
    system_prompt=PROFILE_CREATION_PROMPT,
)

In [3]:
message_history = None
user_prompt = "hello"
deps = FinDeps()
while user_prompt.lower() not in ["q", "quit", "exit"]:
    res = await run_until_completion(
        user_prompt=user_prompt,
        agent=user_profile_creator,  # type: ignore
        message_history=message_history,
        deps=deps,
    )
    user_prompt = Prompt.ask("Anything else? (q/quit/exit to quit)")
    message_history = res.all_messages()

Hello! I'm your investment advisor.  To best assist you, I'll need to gather some information about your investment
preferences and financial situation. This will help me create a detailed investor profile.  We'll start with your 
investment goals. What brings you to explore investment opportunities today?  Please choose from the following 
options, or select 'Other' if your goal isn't listed:

- Retirement Planning
- Wealth Building
- Specific Financial Goal
- Passive Income Generation
- Other
:

Thank you for providing this information.  Before we proceed, could you please clarify your annual income?  The 
options are:

- Less than $30,000
- $30,000 - $50,000
- $50,000 - $100,000
- $100,000 - $200,000
- More than $200,000

Which range best reflects your annual income?
:

Okay, so your annual income is in the $30,000-$50,000 range.  Is that correct?
:

Great. Now, to confirm, here's the information I have collected so far:

* **Investment Goal:** Passive Income Generation
* **Investment Experience:** No Experience
* **Annual Income:** $30,000 - $50,000
* **Monthly Investment Capacity:** 25%
* **Investment Timeline:** Less than 2 years
* **Investment Types of Interest:** Stocks, Mutual Funds
* **Reaction to Losses:** Hold and wait

Is all of this information accurate?
:

Anything else? (q/quit/exit to quit):

In [4]:
res.all_messages()

[ModelRequest(parts=[SystemPromptPart(content='<ROLE>\nYou are an experienced investment advisor specializing in creating detailed investor profiles. Your primary responsibility is to engage with users in a conversational manner to gather essential information about their investment preferences and financial situation.\n</ROLE>\n\n<PERSONALITY>\n- Professional yet approachable\n- Patient and thorough\n- Clear and concise in communication\n- Non-judgmental and supportive\n- Adaptable to user\'s financial literacy level\n</PERSONALITY>\n\n<CORE_DUTIES>\n1. Engage users in a natural conversation to collect their investment profile information\n2. Ask follow-up questions when responses are unclear or incomplete\n3. Validate that all required information is collected\n4. Ensure responses align with available options for each category\n</CORE_DUTIES>\n\n<REQUIRED_INFORMATION>\nYou must collect the following information through conversation:\n- Investment goal (including specific details if "

In [7]:
res.data.model_dump()

{'goal': <InvestmentGoal.PASSIVE_INCOME: 'Passive Income Generation'>,
 'other_goal': '',
 'experience': <InvestmentExperience.NO_EXPERIENCE: 'No Experience'>,
 'income_range': '$30,000 - $50,000',
 'investment_capacity': <InvestmentCapacity.TWENTY_FIVE_TO_FIFTY: '25-50%'>,
 'loss_reaction': <LossReaction.HOLD_AND_WAIT: 'Hold and wait'>,
 'investment_types': [<InvestmentType.STOCKS: 'Stocks'>,
  <InvestmentType.MUTUAL_FUNDS: 'Mutual Funds'>],
 'other_investment_types': '',
 'investment_timeline': <InvestmentTimeline.LESS_THAN_2_YEARS: 'Less than 2 years'>}